In [268]:
import pandas as pd
import numpy as np

In [269]:
df = pd.read_csv('/Users/amitchandna/Downloads/scripts/usa_1.csv', names=["Score", "Team_2", "Result_team_1", "Date","District","Time","Team_1","Team_1_Mascot","Address","Level","Season"], low_memory=False)





In [270]:
s = df.Score
a = df.Address
d = df.District

In [271]:
scored = s.str.split('-', expand=True)
addressed= a.str.split(',', expand=True)
districted = d.str.split('•', expand=True)

In [272]:
df = pd.concat([df,scored], axis=1)

In [273]:
df = df.rename(columns={0:'Score_1', 1:'Score_2'})
df = df.drop(columns={'Score'})

In [274]:
df = pd.concat([df,addressed], axis=1)

In [275]:
df = df.rename(columns={0:'team_1_physical_Address',1:'team_1_City',2:'team_1_State',3:'Team_1_Zip_code'})
df = df.drop(columns={'Address',4,5})

In [276]:
df = pd.concat([df,districted], axis=1)

In [277]:
df = df.rename(columns={0:"Home_Away", 1:'District_non_district'})
df = df.drop(columns={'District'})

In [278]:
df = df[['Team_1','Score_1', 'Team_2', 'Score_2', 'Result_team_1','Home_Away','Team_1_Mascot','Date','District_non_district','Time','Level','Season', 'team_1_physical_Address','team_1_City','team_1_State','Team_1_Zip_code' ]]

In [279]:
df.to_csv('usa_boys_football_1718_clean.csv')

In [280]:
df = df.drop_duplicates()


In [308]:
team_1_score = []
team_2_score = []
for index,row in df.iterrows():
    if row['Score_1'] == None or row['Score_2'] == None:
        row['Score_1'] = -1
        row['Score_2'] = -2
        continue
    numeric_score = ''
    for element in row['Score_1']:
        if element.isdigit():
            numeric_score += element
    if numeric_score != '':
        row['Score_1'] = int(numeric_score)
    else:
        row['Score_1'] = -3
    numeric_score = ''
    for element in row['Score_2']:
        if element.isdigit():
            numeric_score += element
    if numeric_score != '':
        row['Score_2'] = int(numeric_score)
    else:
        row['Score_2'] = -4



In [309]:
type(df.Score_1[4])

str

In [251]:
level = []
season = []
for index,row in df.iterrows():
    if row['Level'] == np.nan:
        level.append('Varsity')
        season.append('17-18')
    else:
        level.append('Varisty')
        season.append('17-18')
    

In [252]:
df['Level'] = level
df['Season'] = season
df.Score_1.value_counts()

42       2952
35       2448
28       2203
49       2152
41       1643
         ... 
86          4
1           4
83          4
90          2
(DFF)       1
Name: Score_1, Length: 88, dtype: int64

In [265]:
for index,row in df.iterrows():
    if row['Result_team_1'] == 'W':
        team_1_score.append(row["Score_1"] if (row["Score_1"] > row["Score_2"]) else row["Score_2"])
        team_2_score.append(row["Score_1"] if (row["Score_1"] < row["Score_2"]) else row["Score_2"])
        continue
    else:
        team_2_score.append(row["Score_1"] if (row["Score_1"] > row["Score_2"]) else row["Score_2"])
        team_1_score.append(row["Score_1"] if (row["Score_1"] < row["Score_2"]) else row["Score_2"])
        continue

In [267]:
df['Score_1'] = team_1_score
df['Score_2'] = team_2_score

ValueError: Length of values does not match length of index

In [168]:
location = ['Home','TBA']

In [169]:
newdf = df[df['Home_Away'] != 'Away']

In [170]:
val = df['Home_Away'].dropna()

In [264]:
type(df.Score_1[3])

int

In [127]:
df.to_csv('usa_football_boys_1718_clean_1.csv')

In [266]:
df

,Team_1,Score_1,Team_2,Score_2,Result_team_1,Home_Away,Team_1_Mascot,Date,District_non_district,Time,Level,Season,team_1_physical_Address,team_1_City,team_1_State,Team_1_Zip_code
0,Mater Dei,31,Bishop AmatLa Puente,7,W,Away,Monarchs,"Fri, 8/25",Non-League,7:00pm,Varisty,17-18,1202 W Edinger Ave,Santa Ana,CA,92707-2191
1,Mater Dei,35,"Bishop GormanLas Vegas, NV",21,W,Home,Monarchs,"Fri, 9/1",Non-League,7:00pm,Varisty,17-18,1202 W Edinger Ave,Santa Ana,CA,92707-2191
2,Mater Dei,42,La Mirada,0,W,Home,Monarchs,"Fri, 9/8",Non-League,7:00pm,Varisty,17-18,1202 W Edinger Ave,Santa Ana,CA,92707-2191
3,Mater Dei,62,"Bergen CatholicOradell, NJ",14,W,TBA,Monarchs,"Sat, 9/16",Non-League,7:30pm,Varisty,17-18,1202 W Edinger Ave,Santa Ana,CA,92707-2191
4,Mater Dei,52,St. Mary'sStockton,14,W,TBA,Monarchs,"Sat, 9/23",Non-League,7:00pm,Varisty,17-18,1202 W Edinger Ave,Santa Ana,CA,92707-2191
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88102,Rossville Christian Academy,34,Memphis NighthawksArlington,12,L,NaN,Wolves,"Fri, 9/22",NaN,5:00pm,Varisty,17-18,280 High St,Rossville,TN,38066
88103,Rossville Christian Academy,55,Nashville ChristianNashville,14,L,NaN,Wolves,"Fri, 9/29",NaN,7:00pm,Varisty,17-18,280 High St,Rossville,TN,38066
88104,Rossville Christian Academy,50,Jackson ChristianJackson,0,L,NaN,Wolves,"Fri, 10/6",NaN,7:00pm,Varisty,17-18,280 High St,Rossville,TN,38066
88105,Rossville Christian Academy,42,Fayette AcademySomerville,0,L,NaN,Wolves,"Fri, 10/20",NaN,7:00pm,Varisty,17-18,280 High St,Rossville,TN,38066
